In [1]:
## Load modules
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import os
cwd = os.getcwd() # current working directory

/home/mhowland/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys
sys.path.append(os.path.abspath(cwd))

In [3]:
## Load data
#data_dir = "cycle_10bins_conditions_none"#"2sec_data_avg" # name of the data folder

# Loading the train, dev, test data from specified folder
#os.chdir(cwd+"/"+data_dir+"/train")
#x_train = genfromtxt("x_train.csv", delimiter=',')
#y_train = genfromtxt("y_train.csv", delimiter=',')

In [8]:
# data (num_samples, features)
basedir = '/home/mhowland/Dropbox/Research/Stanford/Lele/AWS_2016-06_Multi-Height_Time_Series/'
windFile = 'Firewheel_FWL'+str(1)+'_mast_file_tsec_72-months.csv'
def importWindFile(basedir, windFile):
    data = np.loadtxt(basedir+windFile, delimiter=',', skiprows=7)
    ws = np.zeros((data.shape[0],5))
    time = np.linspace(0., 10.*data.shape[0],data.shape[0])
    nt = ws.shape[0]
    hour = data[:,3]
    month = data[:,1]
    for i  in range(0,5):
        ind = 5*(i+1)
        ws[:,i] = data[:,ind]
    return ws, hour, time, nt, month
ws, hour, time, nt, month = importWindFile(basedir, windFile)


In [9]:
n_train = 1000-6
x_train = np.zeros((5,n_train))
y_train = np.zeros((n_train,1))
dataHeight = 0 # index of the height we want
for i in range(6,n_train):
    x_train[:,i] = ws[i-6:i-1,dataHeight]
    y_train[i] = ws[i,dataHeight]

In [6]:
#print(x_all.shape)
#x_train = x_all[10:110,10:20] # (100,10)
data_size, features = x_train.shape

In [7]:
def normalize_data(data):
    mu = np.mean(data,axis=0) # compute the mean along axis = 0 (num_samples for raw data)
    cov = np.std(data,axis=0) # using std instead of variance seems to be best
    return mu, cov # returning the normalizations for the data

x_mu, x_cov = normalize_data(x_train) # x_train is (features x num_samples)
y_mu, y_cov = normalize_data(x_train) # x_train is (features x num_samples)

#y_mu, y_cov = normalize_data(y_train)

X_train = ((x_train - x_mu)/x_cov).T # still in (features, data_samples)
Y_train = ((y_train - y_mu)/x_cov).T # still in (features, data_samples)
#Y_train = 0.5*X_train[1,:].T
print(Y_train.shape)
#print(np.mean(X_train[:,1]))
#X_dev = ((x_dev - x_mu)/x_cov).T # Use same distrib for others --> don't use "future" data
#X_test = ((x_test - x_mu)/x_cov).T

MemoryError: 

In [ ]:
def initialize_parameters(num_hid_layers, size_hid_layers, n_x, output_size):
    parameters = {}
    total_layers = num_hid_layers+1
    
    for l in range(1,total_layers+1):
        if l == 1:
            a = size_hid_layers
            b = n_x
        elif l == total_layers:
            a = output_size
            b = size_hid_layers
        else:
            a = size_hid_layers
            b = size_hid_layers
            
        parameters['w' + str(l)] = tf.get_variable('w'+str(l), [a, b], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
        parameters['b' + str(l)] = tf.get_variable('b'+str(l), [a,1], initializer = tf.zeros_initializer())    
    return parameters

In [ ]:
def forward_prop(X,parameters):
    total_layers = len(parameters)//2
    layer_outputs = {}
    layer_outputs['A0'] = X
    
    for l in range(1,total_layers+1):
        layer_outputs['Z' + str(l)] = tf.matmul(parameters['w' + str(l)],layer_outputs['A' + str(l-1)])+parameters['b' + str(l)]
        layer_outputs['A' + str(l)] = tf.nn.relu(layer_outputs['Z' + str(l)])
        #layer_outputs['A' + str(l)] = tf.nn.dropout(layer_outputs['A' + str(l)],keep_prob)
    
    return layer_outputs['Z' + str(total_layers)]

In [ ]:
def model(X_train, Y_train, learning_rate, num_epochs, num_hid_layers, size_hid_layers, minibatch_size, print_interval=10):

    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    losses = []                                        # To keep track of the cost
    
    O = 1 # output size?
    
    X = tf.placeholder(tf.float32,[n_x,None])
    Y = tf.placeholder(tf.float32,[None])
    
    parameters = initialize_parameters(num_hid_layers, size_hid_layers, n_x, O)
    out = forward_prop(X, parameters)
    
    loss = tf.reduce_mean(tf.squared_difference(out, Y)) # L2 loss --> not good for our problem
    #loss = tf.reduce_mean(tf.losses.absolute_difference(Y,tf.squeeze(out))) # L1 loss
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss) # Optimizer, change the learning rate here

    init = tf.global_variables_initializer() # When init is run later (session.run(init)),
    with tf.Session() as sess: # starting tf session --> all computation on tf graph in this with struct
        sess.run(init)
        for epoch in range(num_epochs+1):
            _, loss_val = sess.run([optimizer, loss], feed_dict={X: X_train, Y: Y_train})
            losses.append(loss_val)
            if epoch % (num_epochs/print_interval) == 0:
                print("Loss: ",loss_val)
            

In [ ]:
learning_rate = 0.008#0.0005
num_epochs = 1000 # total number of epochs to iterate through
print_interval = 10 # number of prints per total run
minibatch_size = 10
num_hid_layers = 3
size_hid_layers = 128

model(X_train, Y_train, learning_rate, num_epochs, num_hid_layers, size_hid_layers, minibatch_size, print_interval)